<a href="https://colab.research.google.com/github/shubha045/Project-/blob/main/openai_code_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ⚙️ Step 1: Install necessary packages
!pip install openai-agents

# 🔑 Step 2: Set your OpenAI API key
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get ('OPENAI_API_KEY')

In [ ]:
# 🧰 Step 3: Import tools and define all necessary functions
from agents import Agent, Runner, function_tool
import subprocess

# 🎯 Tool 1: Execute the code using subprocess
@function_tool
def execute_code(code: str) -> dict:
    try:
        result = subprocess.run(
            ['python3', '-c', code],
            capture_output=True,
            timeout=5,
            text=True
        )
        return {
            "stdout": result.stdout,
            "stderr": result.stderr,
            "returncode": result.returncode
        }
    except Exception as e:
        return {"stderr": str(e), "stdout": "", "returncode": -1}

# 🎯 Tool 2: Parse error from stderr
@function_tool
def parse_error(stderr: str) -> str:
    if not stderr:
        return "No errors found."
    return f"Error detected:\n{stderr}"

# 🎯 Tool 3: Suggest fix using OpenAI agent
# Define a correction agent to suggest code fixes
correction_agent = Agent(
    name="Corrector",
    instructions="You are an expert Python developer. You fix code based on the code and error messages provided.",
    model="gpt-4"
)

@function_tool
def suggest_fix(code: str, error: str) -> str:
    prompt = f"""
    I have the following Python code:

    ```python
    {code}
    ```

    And it throws this error:

    ```
    {error}
    ```

    Please suggest how to fix it.
    """
    response = correction_agent(prompt)
    return response.output

In [ ]:
# 🤖 Final Debugger Agent
debugger_agent = Agent(
    name="Debugger",
    instructions="Take Python code, run it, parse errors, and suggest fixes.",
    tools=[execute_code, parse_error, suggest_fix]
)

In [ ]:
code_input = """
def divide(a, b):
    return a / b

print(divide(10, 5))
"""
import asyncio
async def run_debugger():
   result = await Runner.run(starting_agent=debugger_agent, input= code_input)
   print("💡 Final Output:\n", result.final_output)

await run_debugger()

💡 Final Output:
 The code executed successfully and returned `2.0`. There were no errors. If you have any more code to run or need further assistance, feel free to ask!
